# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [37]:
# import libraries
from sqlalchemy import create_engine
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.pipeline import Pipeline
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
import pickle

In [38]:
# load data from database
engine = create_engine('sqlite:///data/DisasterResponse.db')

df = pd.read_sql_table('InsertTableName', engine)
X = df['message']
Y = df.drop(columns=['id', 'original', 'genre', 'message'], axis=1)

### 2. Write a tokenization function to process your text data

In [39]:
def tokenize(text):
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower())
    tokens = nltk.word_tokenize(text)
    tokens = [WordNetLemmatizer().lemmatize(word) for word in tokens if word not in stopwords.words("english")]

    return tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [40]:
pipeline = Pipeline([
                ('vect', CountVectorizer(tokenizer=tokenize)),
                ('tfidf', TfidfTransformer()),
                ('classifier', MultiOutputClassifier(RandomForestClassifier()))
                ])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [41]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3)


In [45]:
Y_train.groupby('related').count()

,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,water,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
related,,,,,,,,,,,,,,,,,,,,,
0,4296,4296,4296,4296,4296,4296,4296,4296,4296,4296,...,4296,4296,4296,4296,4296,4296,4296,4296,4296,4296
1,13923,13923,13923,13923,13923,13923,13923,13923,13923,13923,...,13923,13923,13923,13923,13923,13923,13923,13923,13923,13923


In [43]:
pipeline.fit(X_train,Y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [6]:
category_names=Y.columns
y_pred = pipeline.predict(X_test)
df_y_pred = pd.DataFrame(data=y_pred, columns=Y_test.columns)
for col in category_names:
    print(col,classification_report(Y_test[col], df_y_pred[col], labels=[0, 1]))

related              precision    recall  f1-score   support

          0       0.63      0.46      0.54      1827
          1       0.85      0.91      0.88      5978

avg / total       0.80      0.81      0.80      7805

request              precision    recall  f1-score   support

          0       0.89      0.98      0.93      6473
          1       0.81      0.46      0.58      1392

avg / total       0.88      0.89      0.87      7865

offer              precision    recall  f1-score   support

          0       1.00      1.00      1.00      7830
          1       0.00      0.00      0.00        35

avg / total       0.99      1.00      0.99      7865

aid_related              precision    recall  f1-score   support

          0       0.75      0.85      0.79      4606
          1       0.73      0.59      0.65      3259

avg / total       0.74      0.74      0.74      7865

medical_help              precision    recall  f1-score   support

          0       0.92      0.99      0

C:\Users\YAVUZ\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\YAVUZ\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


### 6. Improve your model
Use grid search to find better parameters. 

In [7]:
parameters =  {
    'classifier__estimator__n_estimators': [20,30]
}

cv = GridSearchCV(pipeline, param_grid=parameters) 
cv.fit(X_train, Y_train)

GridSearchCV(cv=None, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))]),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'classifier__estimator__n_estimators': [20, 30]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [8]:
y_pred_best = cv.predict(X_test)
df_y_pred_best = pd.DataFrame(data=y_pred_best, columns=category_names)
for col in category_names:
    print(col,classification_report(Y_test[col], df_y_pred_best[col], labels=[0, 1]))

related              precision    recall  f1-score   support

          0       0.68      0.44      0.53      1827
          1       0.84      0.93      0.89      5978

avg / total       0.80      0.82      0.80      7805

request              precision    recall  f1-score   support

          0       0.90      0.98      0.94      6473
          1       0.83      0.47      0.60      1392

avg / total       0.89      0.89      0.88      7865

offer              precision    recall  f1-score   support

          0       1.00      1.00      1.00      7830
          1       0.00      0.00      0.00        35

avg / total       0.99      1.00      0.99      7865

aid_related              precision    recall  f1-score   support

          0       0.78      0.84      0.81      4606
          1       0.75      0.66      0.70      3259

avg / total       0.76      0.77      0.76      7865

medical_help              precision    recall  f1-score   support

          0       0.92      1.00      0

C:\Users\YAVUZ\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\YAVUZ\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

In [10]:
pickle.dump(cv, open('models/classifier.pkl', 'wb'))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.